In [2]:
import pandas as pd

In [3]:
# path to manifest file xslx
manifest_file = ""

# TODO add relationships from reference query reults
reference_file = ""

# output file name
dbml_filename = manifest_file.split('.')[0] + ".dbml"

In [4]:
# sheets in manifest excel workbook
is_sheet_name = "sistema"
tables_sheet_name = "lentelės"
columns_sheet_name = "stulpeliai"

In [5]:
df_is = pd.read_excel(manifest_file, sheet_name=is_sheet_name)
df_tables = pd.read_excel(manifest_file, sheet_name=tables_sheet_name)
df_columns = pd.read_excel(manifest_file, sheet_name=columns_sheet_name)

df_ref = pd.read_csv(reference_file, sep=';')

In [6]:
dbml_lines = []
for row_index, row in df_tables.iterrows():
    table_name = row['table'].replace('$', '_')
    header = f"Table {table_name} {{\n"
    
    columns = ""
    for col_index, col in (df_columns[df_columns['table'] == row['table']]).iterrows():
        col_name = col['field'].replace('#', '_').replace('$', '_')

        columns += f"\t{col_name} {col['type']}"
        
        column_comment = str(col['comment']).replace("'", '"')
        column_note = f"note: '{column_comment}'"
        primary_key = "primary key" if col['is_primary'] else False
        
        if primary_key and (column_note != "note: 'nan'"):
            details = f"[{primary_key}, {column_note}]"
        elif primary_key:
            details = f"[{primary_key}]"
        elif (column_note != "note: 'nan'"):
            details = f"[{column_note}]"
        else:
            details = False

        if details:
            columns += f" {details}"
        
        columns += "\n"
    
    table_note = f"\tNote: '''\n\t\t{row['lenteles_paaiskinimas']}\n\t\tObjektas: {row['objektas']}\n\t\tĮrašu skaičius: {row['įrašų skaičius']} \n\t'''\n"
    footer = "}\n"
    dbml_line = header + columns + table_note + footer
    dbml_lines.append(dbml_line)

In [7]:
with open(dbml_filename, 'w', encoding="utf8") as dmbl:
    dmbl.writelines(dbml_lines)